# Advanced Visualization with AlphaGenome

This notebook covers advanced visualization techniques for AlphaGenome predictions.

## What you'll learn:
- Create publication-quality figures
- Multi-panel visualizations
- Custom styling and annotations
- Export to various formats (PNG, PDF, SVG)

---

## 1. Import Libraries and Setup

In [ ]:
# AlphaGenome libraries
from alphagenome.data import genome
from alphagenome.models import dna_client

# Advanced visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
import pandas as pd
from pathlib import Path

# Custom tools
import sys
sys.path.insert(0, '/shared/tools')
from alphagenome_tools import (
    quick_plot,
    plot_variant_comparison,
    plot_expression_heatmap,
    plot_tracks_overlaid,
    create_multi_panel_figure,
    save_figure,
    monitor_api_quota
)

print("✓ All libraries imported successfully")

## 2. Configure Visualization Settings

In [ ]:
# Set style and parameters
sns.set_style('whitegrid')

# Figure settings
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'DejaVu Sans'

# Color palette
colors = {
    'reference': '#4477AA',
    'alternate': '#CC6677',
    'highlight': '#DDCC77',
    'neutral': '#888888'
}

print("✓ Visualization settings configured")
print(f"✓ Using color palette: {list(colors.keys())}")

## 3. Load Example Data for Visualization

In [ ]:
# Connect to AlphaGenome
model = dna_client.create()
print("✓ Connected to AlphaGenome")

# Define a region for visualization
interval = genome.Interval(
    chromosome='chr22',
    start=35677410,
    end=35877410  # 200kb region
)

print(f"\nAnalysis interval: {interval.chromosome}:{interval.start:,}-{interval.end:,}")

## 4. Custom Track Visualization

Create overlaid genomic tracks with custom styling.

In [ ]:
# Generate synthetic track data for demonstration
# In practice, this would come from AlphaGenome predictions

n_points = 1000
x = np.linspace(interval.start, interval.end, n_points)

# Create multiple tracks
track1 = np.sin(np.linspace(0, 4*np.pi, n_points)) * 0.5 + 0.5
track2 = np.cos(np.linspace(0, 4*np.pi, n_points)) * 0.3 + 0.4
track3 = np.random.randn(n_points).cumsum() / 50

tracks_data = {
    'Gene Expression': track1,
    'Chromatin Accessibility': track2,
    'Conservation Score': track3
}

# Plot overlaid tracks
fig = plot_tracks_overlaid(
    tracks_data=tracks_data,
    interval=interval,
    figsize=(16, 6)
)

if fig:
    plt.tight_layout()
    plt.show()
    print("✓ Overlaid tracks visualization created")

## 5. Gene Expression Heatmap

In [ ]:
# Create synthetic expression data for heatmap
# Rows: different genes or genomic regions
# Columns: genomic positions or samples

n_genes = 20
n_positions = 50

# Generate synthetic expression data
expression_data = np.random.randn(n_genes, n_positions)
# Add some structure
for i in range(n_genes):
    expression_data[i, :] += np.sin(np.linspace(0, i, n_positions)) * 2

# Create heatmap
fig = plot_expression_heatmap(
    data=expression_data,
    figsize=(12, 8),
    cmap='viridis',
    title='Gene Expression Heatmap'
)

if fig:
    plt.show()
    print("✓ Expression heatmap created")

## 6. Multi-Panel Publication Figure

Create a complex multi-panel figure suitable for publications.

In [ ]:
# Prepare data for multi-panel figure
# In practice, these would be actual AlphaGenome outputs

panel_data = [
    track1,  # Panel A
    track2,  # Panel B
    expression_data[:10, :],  # Panel C
    track3,  # Panel D
]

panel_titles = [
    'A. Gene Expression',
    'B. Chromatin Accessibility',
    'C. Expression Heatmap',
    'D. Conservation Score'
]

# Create multi-panel figure
fig = create_multi_panel_figure(
    outputs_list=panel_data,
    titles=panel_titles,
    figsize=(16, 12),
    main_title='AlphaGenome Multi-Modal Predictions'
)

if fig:
    plt.tight_layout()
    plt.show()
    print("✓ Multi-panel figure created")

## 7. Custom Styling and Annotations

In [ ]:
# Create a fully custom plot
fig, ax = plt.subplots(figsize=(14, 6))

# Plot data with custom styling
ax.plot(x, track1, color=colors['reference'], linewidth=2, label='Reference')
ax.plot(x, track2, color=colors['alternate'], linewidth=2, label='Alternate')

# Add variant annotation
variant_pos = interval.start + (interval.end - interval.start) // 2
ax.axvline(variant_pos, color=colors['highlight'], linestyle='--', linewidth=2, label='Variant Position')

# Add shaded region
ax.axvspan(variant_pos - 10000, variant_pos + 10000, alpha=0.2, color=colors['highlight'])

# Custom styling
ax.set_xlabel('Genomic Position', fontsize=12, fontweight='bold')
ax.set_ylabel('Prediction Score', fontsize=12, fontweight='bold')
ax.set_title('Custom Styled AlphaGenome Visualization', fontsize=14, fontweight='bold')

# Custom legend
legend = ax.legend(loc='upper right', frameon=True, shadow=True)
legend.get_frame().set_facecolor('white')
legend.get_frame().set_alpha(0.9)

# Add grid
ax.grid(True, alpha=0.3, linestyle=':')

# Add text annotation
ax.text(0.02, 0.98, 'AlphaGenome Analysis',
        transform=ax.transAxes, fontsize=10,
        verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("✓ Custom styled visualization created")

## 8. Export to Multiple Formats

In [ ]:
# Create output directory
output_dir = Path.home() / 'work' / 'results' / 'visualizations'
output_dir.mkdir(parents=True, exist_ok=True)

# Save figure in multiple formats
base_path = output_dir / 'custom_visualization'

formats = ['png', 'pdf', 'svg']
for fmt in formats:
    save_figure(fig, base_path, formats=[fmt])

print(f"✓ Figure saved in {len(formats)} formats:")
for fmt in formats:
    print(f"  - {base_path}.{fmt}")

print(f"\nAll visualizations saved to: {output_dir}")

## 9. Advanced: Subplot Grid with Shared Axes

In [ ]:
# Create complex subplot grid
fig = plt.figure(figsize=(16, 10))

# Create grid specification
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# Main plot (spans 2 columns)
ax_main = fig.add_subplot(gs[0, :])
ax_main.plot(x, track1, color=colors['reference'], linewidth=2)
ax_main.set_title('Main Prediction Track', fontsize=12, fontweight='bold')
ax_main.set_ylabel('Score')

# Subplot 1
ax1 = fig.add_subplot(gs[1, 0])
ax1.plot(x, track2, color=colors['alternate'], linewidth=1.5)
ax1.set_title('Track 1')
ax1.sharex(ax_main)

# Subplot 2
ax2 = fig.add_subplot(gs[1, 1])
ax2.plot(x, track3, color=colors['highlight'], linewidth=1.5)
ax2.set_title('Track 2')
ax2.sharex(ax_main)

# Heatmap
ax3 = fig.add_subplot(gs[2, :])
im = ax3.imshow(expression_data[:10, :], aspect='auto', cmap='viridis')
ax3.set_title('Expression Summary')
plt.colorbar(im, ax=ax3)

# Overall title
fig.suptitle('Complex AlphaGenome Visualization', fontsize=16, fontweight='bold', y=0.995)

plt.show()

# Save
fig.savefig(output_dir / 'complex_visualization.png', dpi=300, bbox_inches='tight')
print("✓ Complex subplot visualization created and saved")

## 10. Color Palette and Theme Selector

In [ ]:
# Define publication-ready color palettes
palettes = {
    'nature': ['#E64B35', '#4DBBD5', '#00A087', '#3C5488', '#F39B7F'],
    'science': ['#3C5488', '#F39B7F', '#8491B4', '#91D1C2', '#DC0000'],
    'colorblind': ['#0072B2', '#009E73', '#D55E00', '#CC79A7', '#F0E442']
}

# Test different palettes
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, (name, palette) in enumerate(palettes.items()):
    for i, color in enumerate(palette):
        axes[idx].bar(i, 1, color=color, label=f'Color {i+1}')
    axes[idx].set_title(name.capitalize() + ' Palette', fontweight='bold')
    axes[idx].set_xticks(range(len(palette)))
    axes[idx].set_xticklabels([f'C{i+1}' for i in range(len(palette))])
    axes[idx].set_ylim(0, 1.2)

plt.tight_layout()
plt.show()

print("Available color palettes:")
for name in palettes.keys():
    print(f"  - {name}")

## Summary

In this notebook, you learned advanced visualization techniques:

✓ Custom track visualization  
✓ Gene expression heatmaps  
✓ Multi-panel publication figures  
✓ Custom styling and annotations  
✓ Export to multiple formats (PNG, PDF, SVG)  
✓ Complex subplot grids  
✓ Publication-ready color palettes  

### Best Practices:
- Use high DPI (300+) for publication figures
- Save in vector format (PDF/SVG) when possible
- Use colorblind-friendly palettes
- Always include clear labels and legends
- Test color schemes in grayscale

### Resources:
- Matplotlib documentation: https://matplotlib.org/
- Seaborn gallery: https://seaborn.pydata.org/examples/
- ColorBrewer palettes: https://colorbrewer2.org/